In [3]:
!pip install biopython
!pip install SeqIO
#fasta parser

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 29.2 MB/s eta 0:00:00
  Using cached seqio-0.0.19-py3-none-any.whl.metadata (51 kB)
  Using cached clu-0.0.12-py3-none-any.whl.metadata (1.9 kB)
  Using cached jax-0.4.35-py3-none-any.whl.metadata (22 kB)
  Using cached pyglove-0.4.4-py3-none-any.whl.metadata (6.7 kB)
  Using cached tfds_nightly-4.9.2.dev202308090034-py3-none-any.whl.metadata (9.3 kB)
  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
  Using cached etils-1.10.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached promise-2.3.tar.gz (19 kB)
  Preparing metadata (setup.py) ... done
  Using cached tensorflow_metadata-1.16.1-py3-none-any.whl.metadata (2.4 kB)
  Using cached toml-0.10.2-py2.py3-none-any.whl.metadata (7.1 kB)
  Using cached flax-0.10.2-py3-none-any.whl.metadata (11 kB)
  Using cached ml_collections-1.0.0-py3-none-any.whl.metadata (22 kB)
  Using cached docstring_parser-0.16-py3-none-any.whl.metadata (3.0 kB)
  Using cached fsspec-2024.10.0-py3-n

Process FASTA Files

Create DataFrame

Read in file 

Extract Gene Name / ID and set as primary key

Add chromosome # to DF

Add location to DF

Add regenerative label (1)

Save nucleotide as a string

In [4]:
import os
from Bio import SeqIO
import pandas as pd

# Directory containing the dataset
base_dir = "./dataset"

# Function to parse a single .fna file and extract sequence
def parse_fna(file_path):
    sequences = []
    for record in SeqIO.parse(file_path, "fasta"):
        sequences.append(str(record.seq))
    return sequences

def get_header_as_list(file_path):
    with open(file_path, "r") as file:
        for line in file:
            if line.startswith(">"):  # Check if the line is a header
                return line.strip().split(" ")  # Strip newline and split by space
    return []

# Process all files in the dataset directory
data = []

for root, dirs, files in os.walk(base_dir):
    for file_name in files:
        if file_name.endswith(".fna"):
            # Determine folder name (e.g., 'regen' or 'non-regen')
            folder_name = os.path.basename(root)
            regen_value = 1 if folder_name.lower() == "regen" else 0  # regen: 1, non-regen: 0

            # Parse file
            gene_name = file_name.replace("axolotl_", "").replace("_.fna", "")
            file_path = os.path.join(root, file_name)
            sequences = parse_fna(file_path)

            first_line = get_header_as_list(file_path)

            organism = first_line[2].split("=")[1] + " " + first_line[3].strip("]")
            gene_id = first_line[4].split("=")[1].strip("]")
            chromosome = first_line[5].split("=")[1].strip("]")

            for seq in sequences:
                data.append({"gene_name": gene_name, "sequence": seq, "organism": organism, "gene_id": gene_id,"chromosome": chromosome, "regen": regen_value})

# Convert to DataFrame
df = pd.DataFrame(data)
print(f"Processed {len(df)} sequences.")
print(df.head())

# Save as a CSV for feature engineering
df.to_csv("processed_genes.csv", index=False)


Processed 81 sequences.
  gene_name                                           sequence  \
0    HOXA13  GGGGGCGGGCTGCCGCGCCAGCCCCGCTAAGGCTCGGCGGAGAGGG...   
1    TWIST3  GAGAGAAAAAGCCGAAGTGGTGTGAAGCATTTTGGAAGTCTTAGAT...   
2    HOXD10  ATTCTTTTTAATTTATATTAATAAATTGCACCACCCTTTGCACCTG...   
3    HOXD11  GCAGTGGTTTGAAGGAGGCTGGGTTTGTTTATTTATTTCGAATACC...   
4    TGFBR1  GTACAGCCAATGACCCGGGGTGAGGGGGGATGAGGGGTTCAGGATC...   

              organism    gene_id chromosome  regen  
0  Ambystoma mexicanum  138569852         2q      1  
1  Ambystoma mexicanum  138582160         3q      1  
2  Ambystoma mexicanum  138488809          9      1  
3  Ambystoma mexicanum  138488810          9      1  
4  Ambystoma mexicanum  138497249         5q      1  
